This notebook is a temporary holding place for code that we are testing but not ready to implement immediately

In [ ]:
# test gtn calculation using generator id

# load the allocated eia data for each month where there is corresponding cems data
eia_plant_month_net_gen = gen_fuel_allocated[(gen_fuel_allocated['data_source'] == 'cems') & ~(gen_fuel_allocated['net_generation_mwh'].isna())]
# crosswalk the generator_id into the cems data
generator_crosswalk = data_cleaning.get_epa_eia_crosswalk()
unique_unit_to_generator = generator_crosswalk.groupby(['plant_id_eia','unitid'])['generator_id'].count().reset_index()
unique_unit_to_generator = unique_unit_to_generator[unique_unit_to_generator['generator_id'] == 1].drop(columns='generator_id')

unique_unit_to_generator = unique_unit_to_generator.merge(generator_crosswalk[['plant_id_eia','unitid','generator_id']], how='left', on=['plant_id_eia','unitid'])

cems_plant_month_gross_gen = cems.merge(unique_unit_to_generator, how='inner', on=['plant_id_eia','unitid'])
cems_plant_month_gross_gen = cems_plant_month_gross_gen.groupby(['plant_id_eia','generator_id','report_date']).sum()['gross_generation_mwh'].reset_index()

# merge the net generation data into the gross generation data
monthly_gtn_ratio = cems_plant_month_gross_gen.merge(eia_plant_month_net_gen, how='left', on=['plant_id_eia','report_date'])

# calculate the gtn
monthly_gtn_ratio['gross_to_net_ratio'] = monthly_gtn_ratio['net_generation_mwh'] / monthly_gtn_ratio['gross_generation_mwh']

## Outlier Detection

In [ ]:
max = cems.replace(0, np.NaN).groupby(['cems_id'])['heat_content_mmbtu'].max()
mean = cems.replace(0, np.NaN).groupby(['cems_id'])['heat_content_mmbtu'].mean()
stdev = cems.replace(0, np.NaN).groupby(['cems_id'])['heat_content_mmbtu'].std()

max[max > mean + (3 * stdev)]

px.line(cems[cems['cems_id'] == '10298_CG803'], x='operating_datetime_utc', y='heat_content_mmbtu')

px.line(cems[cems['cems_id'] == '1012_2'], x='operating_datetime_utc', y='heat_content_mmbtu')